In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import gc
import pandas as pd
import lightgbm as lgb
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score, auc
from sklearn.model_selection import KFold
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from sklearn.model_selection import StratifiedKFold
import category_encoders
from catboost import Pool
from catboost.utils import get_roc_curve
from sklearn.metrics import auc


In [2]:
train_data = pd.read_csv('./train_dataset/train_public.csv')
test_public = pd.read_csv('./test_public.csv')
train_internet = pd.read_csv('./train_dataset/train_internet.csv', encoding='gb2312')

In [3]:
train_internet

,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,sub_class,work_type,employer_type,...,earlies_credit_mon,title,policy_code,f0,f1,f2,f3,f4,f5,is_default
0,119262,0,12000,5,11.53,264.10,B,B5,职员,普通企业,...,1984/3/1,0.0,1,1.0,0.0,8.0,17.0,8.0,1.0,1
1,369815,1,8000,3,13.98,273.35,C,C3,其他,普通企业,...,1992/1/1,94.0,1,NaN,NaN,NaN,NaN,NaN,NaN,0
2,787833,2,20000,5,17.99,507.76,D,D2,工人,上市企业,...,1996/10/1,0.0,1,6.0,0.0,10.0,8.0,3.0,0.0,0
3,671675,3,10700,3,10.16,346.07,B,B1,职员,普通企业,...,2000/7/1,41646.0,1,3.0,0.0,4.0,11.0,6.0,0.0,0
4,245160,4,8000,3,8.24,251.58,B,B1,其他,政府机构,...,2000/3/1,4.0,1,3.0,0.0,8.0,6.0,4.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
749995,624287,749995,12000,3,11.47,395.55,B,B5,职员,上市企业,...,1995/6/1,0.0,1,6.0,0.0,8.0,22.0,12.0,5.0,0
749996,427602,749996,12000,3,6.03,365.23,A,A1,工人,政府机构,...,2001/9/1,4.0,1,2.0,0.0,7.0,4.0,4.0,0.0,0
749997,206828,749997,10000,3,15.41,348.67,D,D1,职员,政府机构,...,2008/10/1,8.0,1,6.0,0.0,10.0,11.0,3.0,0.0,0
749998,293912,749998,7200,3,9.44,230.44,B,B1,其他,政府机构,...,2006/4/1,0.0,1,7.0,0.0,9.0,11.0,6.0,0.0,0


In [4]:
X_train = train_data.copy()
X_test = test_public.copy()
X_train = X_train.drop(['isDefault'], axis = 1)
X_train.shape
X_train['is_train'] = 1
X_test['is_train'] = 0

X_traintest = X_traintest = pd.concat([X_train, X_test], axis = 0)
X_traintest.shape
X_internet = train_internet.copy()

### 数据探索 

In [5]:
unimport_fea = ['employer_type', 'industry', 'region', 'policy_code', 'pub_dero_bankrup', 'f1', 'del_in_18month', 'initial_list_status']
for cat in unimport_fea:
    X_internet = X_internet.drop(cat, axis=1)

In [6]:
unimport_fea = ['employer_type', 'industry', 'region', 'policy_code', 'pub_dero_bankrup', 'f1', 'del_in_18month', 'initial_list_status']
for cat in unimport_fea:
    X_traintest = X_traintest.drop(cat, axis=1)

### 特征工程 

#### work_year, class 

In [7]:
work_year_dict = {
    '< 1 year': 0,
    '1 year': 1,
    '2 years': 2,
    '3 years': 3,
    '4 years': 4,
    '5 years': 5,
    '6 years': 6,
    '7 years': 7,
    '8 years': 8,
    '9 years': 9,
    '10+ years': 10,
}


X_traintest['work_year'] = X_traintest['work_year'].map(work_year_dict)
X_internet['work_year'] = X_internet['work_year'].map(work_year_dict)

In [8]:
class_dict = {
    'A': 1,
    'B': 2,
    'C': 3,
    'D': 4,
    'E': 5,
    'F': 6,
    'G': 7,
}


X_traintest['class'] = X_traintest['class'].map(class_dict)
X_internet['class'] = X_internet['class'].map(class_dict)

#### issue_date, earlies_credit_mon

In [9]:
X_traintest['issue_date'] = pd.to_datetime(X_traintest['issue_date'])
X_internet['issue_date'] = pd.to_datetime(X_internet['issue_date'])

X_traintest['issue_date_delta'] = (X_traintest['issue_date'] - X_traintest['issue_date'].min())  / np.timedelta64(1,'D')
X_internet['issue_date_delta'] = (X_internet['issue_date'] - X_internet['issue_date'].min())  / np.timedelta64(1,'D')

In [10]:
X_traintest['earlies_credit_mon'] = pd.to_datetime(X_traintest['earlies_credit_mon'])
X_internet['earlies_credit_mon'] = pd.to_datetime(X_internet['earlies_credit_mon'])

X_traintest['earlies_credit_mon_delta'] = (X_traintest['earlies_credit_mon'] - X_traintest['earlies_credit_mon'].min())  / np.timedelta64(1,'D')
X_internet['earlies_credit_mon_delta'] = (X_internet['earlies_credit_mon'] - X_internet['earlies_credit_mon'].min())  / np.timedelta64(1,'D')

In [11]:
X_traintest['issue_earlies_diff'] = (X_traintest['issue_date'] - X_traintest['earlies_credit_mon'])  / np.timedelta64(1,'D')
X_internet['issue_earlies_diff'] = (X_internet['issue_date'] - X_internet['earlies_credit_mon'])  / np.timedelta64(1,'D')

In [12]:
col_to_drop = ['issue_date', 'earlies_credit_mon']
X_traintest = X_traintest.drop(col_to_drop, axis=1)
X_internet = X_internet.drop(col_to_drop, axis=1)

#### employer_type, industry

#### 交叉特征

In [13]:
X_traintest['pro'] = X_traintest['interest'].mul(X_traintest['year_of_loan'])
X_traintest['loan_year'] = X_traintest['total_loan']/X_traintest['year_of_loan']
X_traintest['early_ratio'] = X_traintest['early_return_amount']/X_traintest['total_loan']
X_traintest['early_times_ratio'] = X_traintest['early_return']/X_traintest['year_of_loan']
X_traintest['recircle_ratio'] = X_traintest['recircle_b']/X_traintest['total_loan']
X_traintest['recircle_amt'] = X_traintest['recircle_b'].mul(X_traintest['total_loan'])

In [14]:
X_internet['pro'] = X_internet['interest'].mul(X_internet['year_of_loan'])
X_internet['loan_year'] = X_internet['total_loan']/X_internet['year_of_loan']
X_internet['early_ratio'] = X_internet['early_return_amount']/X_internet['total_loan']
X_internet['early_times_ratio'] = X_internet['early_return']/X_internet['year_of_loan']
X_internet['recircle_ratio'] = X_internet['recircle_b']/X_internet['total_loan']
X_internet['recircle_amt'] = X_internet['recircle_b'].mul(X_internet['total_loan'])

In [15]:
numerical_fea = list(X_traintest.select_dtypes(exclude=['object']).columns) 
category_fea = list(filter(lambda x: x not in numerical_fea,list(X_traintest.columns)))
# print("numerical_fea:",numerical_fea)
# print("category_fea:",category_fea)

### 模型 

In [16]:
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import catboost

In [17]:
all_fea = X_traintest.columns

In [18]:
X_train = X_traintest[X_traintest.is_train == 1][all_fea]
y_train = train_data['isDefault']

X_test = X_traintest[X_traintest.is_train == 0][all_fea]


In [19]:
X_train[category_fea] = X_train[category_fea].astype(str)
X_test[category_fea] = X_test[category_fea].astype(str)

X_train[numerical_fea] = X_train[numerical_fea].astype(float)
X_test[numerical_fea] = X_test[numerical_fea].astype(float)

In [20]:
model = catboost.CatBoostClassifier(cat_features = category_fea, random_state = 2080)
model.fit(X_train, y_train, verbose=400)

Learning rate set to 0.027538
0:	learn: 0.6597156	total: 168ms	remaining: 2m 47s
400:	learn: 0.2426222	total: 2.15s	remaining: 3.21s
800:	learn: 0.1938693	total: 3.9s	remaining: 969ms
999:	learn: 0.1753391	total: 5.04s	remaining: 0us


In [21]:
y_pred = model.predict_proba(X_test)

### 模型改进

In [22]:
feature_importances = model.get_feature_importance(prettified=True,
                                                   thread_count=-1,
                                                   verbose=False)

In [23]:
tr_cols = set(X_traintest.columns)
same_col = list(tr_cols.intersection(set(X_internet.columns)))

tr_cols.difference(set(same_col))

{'app_type', 'is_train', 'known_dero', 'known_outstanding_loan'}

In [24]:
for i in range(feature_importances['Feature Id'].size):
    if (feature_importances['Feature Id'][i] in same_col):
        if feature_importances['Importances'][i]>1:
            X_traintest[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_traintest[feature_importances['Feature Id'][i]])
            X_internet[feature_importances['Feature Id'][i]+'_cnt'] = category_encoders.count.CountEncoder().fit_transform(X_internet[feature_importances['Feature Id'][i]])


In [25]:
all_fea = X_traintest.columns
X_train = X_traintest[X_traintest.is_train == 1][all_fea]
X_train = pd.concat([X_train, train_data['isDefault']], axis=1)
y_train = train_data['isDefault']

X_test = X_traintest[X_traintest.is_train == 0][all_fea]

X_train = X_train.drop('is_train', axis=1)
X_test = X_test.drop('is_train', axis=1)

In [26]:
tr_cols = set(X_train.columns)
same_col = list(tr_cols.intersection(set(X_internet.columns)))
train_inteSame = X_internet[same_col].copy()

Inte_add_cos = list(tr_cols.difference(set(same_col)))
for col in Inte_add_cos:
    train_inteSame[col] = np.nan


In [27]:
train_inteSame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 65 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   debt_loan_ratio               749770 non-null  float64
 1   post_code_cnt                 749999 non-null  float64
 2   total_loan_cnt                750000 non-null  int64  
 3   user_id_cnt                   750000 non-null  int64  
 4   f0                            712202 non-null  float64
 5   loan_id                       750000 non-null  int64  
 6   house_exist                   750000 non-null  int64  
 7   interest_cnt                  750000 non-null  float64
 8   recircle_ratio                750000 non-null  float64
 9   user_id                       750000 non-null  int64  
 10  f2                            712202 non-null  float64
 11  work_year                     706153 non-null  float64
 12  scoring_high_cnt              750000 non-nul

# lgbm

In [28]:
def train_model(data_, test_, y_, folds_):
    oof_preds = np.zeros(data_.shape[0])
    sub_preds = np.zeros(test_.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in data_.columns if f not in ['loan_id', 'user_id', 'isDefault'] ]
    for n_fold, (trn_idx, val_idx) in enumerate(folds_.split(data_)):
        trn_x, trn_y = data_[feats].iloc[trn_idx], y_.iloc[trn_idx]
        val_x, val_y = data_[feats].iloc[val_idx], y_.iloc[val_idx]
        clf = LGBMClassifier(
            n_estimators=4000,
            learning_rate=0.08,
            num_leaves=2**5,
            colsample_bytree=.65,
            subsample=.9,
            max_depth=5,
#             max_bin=250,
            reg_alpha=.3,
            reg_lambda=.3,
            min_split_gain=.01,
            min_child_weight=2,
            silent=-1,
            verbose=-1,
        )
        
        clf.fit(trn_x, trn_y, 
                eval_set= [(trn_x, trn_y), (val_x, val_y)], 
                eval_metric='auc', verbose=100, early_stopping_rounds=50  #30
               )

        oof_preds[val_idx] = clf.predict_proba(val_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_[feats], num_iteration=clf.best_iteration_)[:, 1] / folds_.n_splits
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(val_y, oof_preds[val_idx])))
        del clf, trn_x, trn_y, val_x, val_y
        gc.collect()
        
    print('Full AUC score %.6f' % roc_auc_score(y_, oof_preds)) 
    
    test_['isDefault'] = sub_preds

    return oof_preds, test_[['loan_id', 'isDefault']], feature_importance_df

In [29]:
folds = KFold(n_splits=5, shuffle=True, random_state=546789)
oof_preds, IntePre, importances = train_model(X_train, train_inteSame, y_train, folds)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[25]	training's auc: 0.918123	training's binary_logloss: 0.278704	valid_1's auc: 0.892969	valid_1's binary_logloss: 0.299946
Fold  1 AUC : 0.892969
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.956981	training's binary_logloss: 0.221484	valid_1's auc: 0.890822	valid_1's binary_logloss: 0.286919
Early stopping, best iteration is:
[61]	training's auc: 0.938812	training's binary_logloss: 0.244966	valid_1's auc: 0.893429	valid_1's binary_logloss: 0.284725
Fold  2 AUC : 0.893429
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[34]	training's auc: 0.927792	training's binary_logloss: 0.259354	valid_1's auc: 0.874895	valid_1's binary_logloss: 0.313055
Fold  3 AUC : 0.874895
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.960791	training's binary_logloss: 0.212792	valid_1's auc:

In [30]:
IntePre['isDef'] = train_internet['is_default']
from sklearn.metrics import roc_auc_score
roc_auc_score(IntePre['isDef'],IntePre.isDefault)

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


0.7318581966166361

In [31]:
InteId = IntePre.loc[IntePre.isDefault<0.02, 'loan_id'].tolist()

X_train['dataSourse'] = 1
X_test['dataSourse'] = 1
train_inteSame['dataSourse'] = 0
train_inteSame['isDefault'] = X_internet['is_default']
use_te = train_inteSame[train_inteSame.loan_id.isin( InteId )].copy()
data = pd.concat([ X_train,X_test,use_te]).reset_index(drop=True)


In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58933 entries, 0 to 58932
Data columns (total 66 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   loan_id                       58933 non-null  int64  
 1   user_id                       58933 non-null  int64  
 2   total_loan                    58933 non-null  float64
 3   year_of_loan                  58933 non-null  int64  
 4   interest                      58933 non-null  float64
 5   monthly_payment               58933 non-null  float64
 6   class                         58933 non-null  int64  
 7   work_year                     56269 non-null  float64
 8   house_exist                   58933 non-null  int64  
 9   censor_status                 58933 non-null  int64  
 10  use                           58933 non-null  int64  
 11  post_code                     58932 non-null  float64
 12  debt_loan_ratio               58921 non-null  float64
 13  s

In [33]:
train = data[data['isDefault'].notna()]
test  = data[data['isDefault'].isna()]




y = train['isDefault']
folds = KFold(n_splits=5, shuffle=True, random_state=546789)
oof_preds, test_preds, importances = train_model(train, test, y, folds)

Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[40]	training's auc: 0.987657	training's binary_logloss: 0.0486178	valid_1's auc: 0.981919	valid_1's binary_logloss: 0.0594187
Fold  1 AUC : 0.981919
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[48]	training's auc: 0.987847	training's binary_logloss: 0.0492981	valid_1's auc: 0.981576	valid_1's binary_logloss: 0.0532253
Fold  2 AUC : 0.981576
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.991231	training's binary_logloss: 0.0434947	valid_1's auc: 0.981575	valid_1's binary_logloss: 0.0539869
Early stopping, best iteration is:
[64]	training's auc: 0.989023	training's binary_logloss: 0.0469012	valid_1's auc: 0.981756	valid_1's binary_logloss: 0.0538436
Fold  3 AUC : 0.981756
Training until validation scores don't improve for 50 rounds
[100]	training's auc: 0.992286	training's binary_logloss: 0.0420619	valid

E:\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


### 输出 

In [34]:
test_preds.rename({'loan_id': 'id'}, axis=1)[['id', 'isDefault']].to_csv('./submit/test_new.csv', index=False)

In [35]:
test_preds


,loan_id,isDefault
10000,1000575,0.010961
10001,1028125,0.076642
10002,1010694,0.005813
10003,1026712,0.005128
10004,1002895,0.010174
...,...,...
14995,1008856,0.328487
14996,1016651,0.014349
14997,1024140,0.005520
14998,1014316,0.012013
